In [30]:
import os
import pandas as pd
from scipy import stats
import lingam

In [6]:
df = pd.read_csv(os.path.join(os.getcwd(), 'data', 'inputs', 'big_monthly_ret_etfs_macro_transf.csv'))
df.set_index('date', inplace=True)

df.head()

,SPY,XLI,XLE,XLK,XLV,XLU,XLF,XLY,XLP,XLB,...,CUSR0000SAC,CUSR0000SAD,CUSR0000SAS,CPIULFSL,CUSR0000SA0L2,CUSR0000SA0L5,PCEPI,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA
date,,,,,,,,,,,,,,,,,,,,,
2000-02-01,-0.021282,-0.079459,0.001764,-0.012479,0.026966,-0.015484,-0.023499,-0.092305,-0.002751,-0.122759,...,0.006116,-0.001597,0.002601,0.004109,0.004902,0.003639,0.003131,-0.002778,0.007243,0.002975
2000-03-01,-0.017010,-0.076506,-0.027588,0.096418,-0.090493,-0.053417,-0.096940,-0.104500,-0.104976,-0.134285,...,0.010111,0.003192,0.003112,0.006423,0.006701,0.006637,0.004292,0.001940,0.012709,0.001775
2000-05-01,-0.032612,0.055099,0.042422,-0.145227,-0.019337,0.079906,-0.004560,0.043250,0.107819,-0.007696,...,0.000000,0.003185,0.002581,0.001164,0.001215,0.001805,0.000836,0.000103,-0.001753,0.001923
2000-06-01,-0.004720,-0.003146,0.071931,-0.062907,-0.024226,0.004376,0.075222,-0.073864,0.042560,-0.021432,...,0.007380,-0.002388,0.004628,0.006957,0.006657,0.005992,0.003390,-0.003126,0.009124,0.002706
2000-08-01,-0.008866,0.019630,-0.040169,-0.056302,0.007473,0.005542,0.058110,0.020346,0.015038,-0.036369,...,-0.004013,-0.000798,0.003061,-0.000577,-0.001203,-0.000596,-0.000064,-0.004124,-0.005135,0.002607


In [31]:
etfs = ["SPY" ,"XLI" ,"XLE" ,"XLK" ,"XLV", "XLU" ,"XLF" ,"XLY" ,"XLP" ,"XLB"]
target = "SPY"
start_window = 12 * 5
p = 2

etfs.remove(target)
target_df = df.drop(etfs, axis=1)

In [33]:
shapiro_wilk_test = stats.shapiro(target_df)
shapiro_wilk_test.pvalue

0.0

In [36]:
target_df.shape

(193, 128)

In [35]:
Xt.shape

(60, 128)

In [38]:
start = 0
end = start + start_window

# for step in range(0, len(target_df) - start_window, 1):

step = 0
Xt = target_df.iloc[start:(end + step), :]

var_lingam = lingam.VARLiNGAM(lags=p)
var_lingam_fit = var_lingam.fit(Xt)

LinAlgError: 60-th leading minor of the array is not positive definite

In [41]:
var_lingam = lingam.VARLiNGAM(lags=p)
var_lingam_fit = var_lingam.fit(Xt)

LinAlgError: 60-th leading minor of the array is not positive definite